## Librairies

In [ ]:
!pip install mat73

In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

import keras
import keras.regularizers as regularizers
from keras import backend as K
from keras.applications import InceptionV3
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,MaxPooling2D,Dropout,AvgPool2D,Activation,BatchNormalization
from sklearn.metrics import accuracy_score, confusion_matrix

In [ ]:
import ipywidgets as widgets
import io
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import skimage
import numpy as np
from PIL import Image
import tqdm
from mat73 import loadmat
from sklearn.model_selection import train_test_split
import cv2
from sklearn.utils import shuffle
import tensorflow as tf

## Custom functions

In [ ]:
# functions below : https://datascience.stackexchange.com/questions/45165/how-to-get-accuracy-f1-precision-and-recall-for-a-keras-model
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [ ]:
def show_confusion_matrix(cm, labels):
  plt.figure(figsize=(10, 8))
  sns.heatmap(cm, xticklabels=labels, yticklabels=labels, 
              annot=True, fmt='g')
  plt.xlabel('Prediction')
  plt.ylabel('Label')
  plt.title('Confusion Matrix')
  plt.show()

def show_confusion_matrix_norm(cm, labels):
  cmn = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
  fig, ax = plt.subplots(figsize=(10,10))
  sns.heatmap(cmn, annot=True, fmt='.2f', xticklabels=label_names, yticklabels=label_names)
  plt.ylabel('Actual')
  plt.xlabel('Predicted')
  plt.title('Confusion Matrix Normalized')
  plt.show(block=False)


label_names = ['Meningioma', 'Glioma', 'Pituitary tumor']

In [ ]:
# https://www.w3schools.com/python/python_ml_auc_roc.asp

import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, roc_curve

def plot_roc_curve(true_y, y_prob):
    """
    plots the roc curve based of the probabilities
    """

    fpr, tpr, thresholds = roc_curve(true_y, y_prob)
    plt.plot(fpr, tpr)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Data

### Lecture des données

In [ ]:
# Sauvegarde des chemins de chacun des .mat
datapath = '/content/drive/MyDrive/Data_SYS843/'
imagespaths = []
for root, dirs, files in os.walk(datapath, topdown=False):
   for name in files:
      imagespaths.append(os.path.join(root, name))

images = []
labels = []
img_size = 299

for path in imagespaths:
  x = loadmat(path)
  img = cv2.resize(x['cjdata']['image'],dsize=(img_size, img_size), interpolation=cv2.INTER_LINEAR)
  cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)
  img = np.uint8(img)
  # img = cv2.equalizeHist(img)
  images.append(img)
  label = x['cjdata']['label']
  label = label.astype(int)-1
  labels.append(label)

labels = np.array(labels)
images = np.array(images)

### Augmentation de données

In [ ]:
from copy import copy
import cv2

def dataAugment(img_input, lbl_input):
  images = []
  labels = []
  for i in range(0,len(img_input)):
    img_temp = copy(img_input[i])
    lbl_temp = lbl_input[i]
    # rotation

    # flip H
    flip_h = cv2.flip(img_temp,1)
    images.append(flip_h)
    labels.append(lbl_temp)

    # flip V
    flip_v = cv2.flip(img_temp,0)
    images.append(flip_v)
    labels.append(lbl_temp)

  return np.array(images), np.array(labels)


(img_augment, lbl_augment) = dataAugment(images,labels)


images = np.concatenate((images,img_augment))
labels = np.concatenate((labels, lbl_augment))

images = np.array(images)
labels = np.array(labels)

### Mélange des données

In [ ]:
X,Y = shuffle(images,labels,random_state=1)

In [ ]:
X_train,X_test,y_train,y_test = train_test_split(X,Y,test_size=0.1,random_state=1)

# W/o data augmentation
# X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.2,random_state=1)

# W/ data augmentation
X_train,X_val,y_train,y_val = train_test_split(X_train,y_train,test_size=0.15,random_state=1)

In [ ]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)
Y = tf.keras.utils.to_categorical(Y, num_classes=3)

## Modèle (CNN)

In [ ]:
cnn = Sequential()
cnn.add(Conv2D(32,(7,7),padding="valid",strides=1,activation='relu',input_shape=(299,299,1)))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(BatchNormalization())
cnn.add(Dropout(0.1))
cnn.add(Conv2D(32,(9,9),padding="same",strides=1,activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(Dropout(0.1))
cnn.add(Conv2D(32,(9,9),padding="same",strides=1,activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(32,(9,9),padding="same",strides=1,activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(64,(11,11),padding="same",strides=1,activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(Dropout(0.2))
cnn.add(Conv2D(64,(11,11),padding="same",strides=1,activation='relu'))
cnn.add(MaxPooling2D(pool_size=(2, 2),strides=1))
cnn.add(Flatten())
cnn.add(Dense(3,activation='softmax'))

In [ ]:
cnn.summary()

In [ ]:
# modele cnn
import keras
from keras import backend as K

epochs = 15
batch_size = 10


callbacks = [
    # keras.callbacks.ModelCheckpoint("/tmp/cnn/save_at_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=3,restore_best_weights=True)
]

# optimizer = keras.optimizers.Adam(1e-3)
optimizer = keras.optimizers.SGD(learning_rate=0.0001,momentum=0.5)

cnn.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy",recall_m,precision_m,f1_m],
)

history = cnn.fit(
    X_train,y_train,
    epochs=epochs,
    validation_data=(X_val,y_val),
    shuffle=True,
    batch_size=batch_size,
    callbacks=callbacks
)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# ACCURACY
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Validation Accuracy")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# LOSS
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# cnn.load_weights("/tmp/cnn/save_at_28.keras")
print("Evaluate on test data")
results = cnn.evaluate(X_test, y_test)
print("test loss, test acc, test mse:", results)

In [ ]:
print("Generate predictions for all tests")
predictions = cnn.predict(X_test)
print("predictions shape:", predictions.shape)
print(y_test.shape)
confusion_mtx_cnn = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))#tf.math.confusion_matrix(labels=tf.argmax(y_test, 1), predictions=tf.argmax(predictions, 1), num_classes=3)

In [ ]:
print("CNN")
show_confusion_matrix(confusion_mtx_cnn, label_names)
show_confusion_matrix_norm(confusion_mtx_cnn, label_names)

## Modèle (InceptionV3)

### InceptionV3 no pretrained with 3 classes output

In [ ]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Input, Conv2D 

# Modèle sans freezing
inception = InceptionV3(weights=None,include_top= 'TRUE', classes= 3) 

input_tensor = Input(shape=(299,299,1) )
x = Conv2D(3,(3,3),padding='same')(input_tensor)    # x has a dimension of (IMG_SIZE,IMG_SIZE,3)
out = inception(x) 
model = Model(inputs=input_tensor,outputs=out)

model.summary()

In [ ]:
# modele no freeze
import keras
from keras import backend as K

epochs = 25
batch_size = 10


callbacks = [
    keras.callbacks.ModelCheckpoint("/tmp/model1/save_at_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)
    # keras.callbacks.ModelCheckpoint(
    #   filepath=checkpoint_filepath_1,
    #   save_weights_only=True,
    #   monitor='val_loss',
    #   mode='min',
    #   save_best_only=True
    #   )
]



# optimizer = keras.optimizers.Adam(1e-3)
optimizer = keras.optimizers.SGD(learning_rate=0.0001,momentum=0.8)

model.compile(
    optimizer=optimizer,
    loss="categorical_crossentropy",
    metrics=["accuracy",recall_m,precision_m,f1_m],
)

history = model.fit(
    X_train,y_train,
    epochs=epochs,
    validation_data=(X_val,y_val),
    shuffle=True,
    batch_size=batch_size,
    callbacks=callbacks
)



In [ ]:
# ACCURACY
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Validation Accuracy")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# LOSS
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# model.load_weights("/tmp/model1/save_at_13.keras")
print("Evaluate on test data")
results = model.evaluate(X_test, y_test)
print("test loss, test acc:", results)

In [ ]:
print("Generate predictions for 3 samples")
predictions = model.predict(X_test)
print("predictions shape:", predictions.shape)

confusion_mtx = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions, axis=1))#tf.math.confusion_matrix(labels=tf.argmax(y_test, 1), predictions=tf.argmax(predictions, 1), num_classes=3)


In [ ]:
print("Model 1")
show_confusion_matrix(confusion_mtx, label_names)
show_confusion_matrix_norm(confusion_mtx, label_names)

### InceptionV3 pretrained + SVM output

In [ ]:
from keras.applications import InceptionV3
from keras.models import Model
from keras.layers import Input, Conv2D

# Modèle avec freezing
inception2 = InceptionV3(weights='imagenet',include_top= False)
inception2.trainable = False
model2 = Sequential()
model2.add(Conv2D(3,(3,3),padding='same',input_shape=(299,299,1)))
model2.add(inception2)
model2.add(Flatten())
model2.add(Dense(3, kernel_regularizer=regularizers.l2(0.01)))
model2.add(Activation('linear'))


model2.summary()

In [ ]:
# modele freeze + SVM
import keras
epochs = 100
batch_size = 10


callbacks = [
    keras.callbacks.ModelCheckpoint("/tmp/model2/save_at_{epoch}.keras"),
    keras.callbacks.EarlyStopping(monitor="val_loss",patience=5,restore_best_weights=True)
    # keras.callbacks.ModelCheckpoint(
    #   filepath=checkpoint_filepath_2,
    #   save_weights_only=True,
    #   monitor='val_loss',
    #   mode='min',
    #   save_best_only=True
    #   )
]


# optimizer = keras.optimizers.SGD(learning_rate=0.0001,momentum=0.9)

model2.compile(
    loss='squared_hinge',
    optimizer='adadelta',
    metrics=['accuracy',recall_m,precision_m,f1_m]
)

history2 = model2.fit(
    X_train,y_train,
    epochs=epochs,
    validation_data=(X_val,y_val),
    shuffle=True,
    batch_size=batch_size,
    callbacks=callbacks
)

In [ ]:
# ACCURACY
acc = history2.history['accuracy']
val_acc = history2.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Validation Accuracy")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# LOSS
loss = history2.history['loss']
val_loss = history2.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(14,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.legend(loc='upper left')
plt.show()

In [ ]:
# model2.load_weights("/tmp/model2/save_at_50.keras")
print("Evaluate on test data")
results = model2.evaluate(X_test, y_test)
print("test loss, test acc:", results)

In [ ]:
print("Generate predictions for 3 samples")
predictions2 = model2.predict(X_test)
print("predictions shape:", predictions2.shape)

confusion_mtx2 = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(predictions2, axis=1))

In [ ]:
print("Model 2")
show_confusion_matrix(confusion_mtx2, label_names)
show_confusion_matrix_norm(confusion_mtx2, label_names)

### To download weights

In [ ]:
# from google.colab import files
# files.download('/tmp/model1/save_at_13.keras')
# files.download('/tmp/model2/save_at_8.keras')
# files.download('/tmp/cnn/save_at_28.keras')